In [ ]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import datetime

In [ ]:
def date_is_valid(date):
    try:
        datetime.date.fromisoformat(date)
    except ValueError as e:
        return False
    return True

def days_from_jan_1st(date):
    try:
        date = datetime.date.fromisoformat(date)
    except ValueError as e:
        print(date)
        raise e
    return (date - datetime.date(date.year, 1, 1)).days

In [ ]:
table = pd.read_csv(
    'decessi_giornalieri.csv', 
    encoding='iso-8859-1',
    dtype={'GE':str})
table = table.loc[(table['MASCHI_20'] != 9999)&(table['FEMMINE_20'] != 9999)]
records = []
for record in table.itertuples():
    record_base = (
        record.NOME_REGIONE, 
        record.NOME_PROVINCIA,
        record.NOME_COMUNE,
        record.CL_ETA)
    day = record.GE[2:4]
    month = record.GE[0:2]
    record_ext = (
        ('M', '2015-' + month + '-' + day, record.MASCHI_15),
        ('M', '2016-' + month + '-' + day, record.MASCHI_16),
        ('M', '2017-' + month + '-' + day, record.MASCHI_17),
        ('M', '2018-' + month + '-' + day, record.MASCHI_18),
        ('M', '2019-' + month + '-' + day, record.MASCHI_19),
        ('M', '2020-' + month + '-' + day, record.MASCHI_20),
        ('F', '2015-' + month + '-' + day, record.FEMMINE_15),
        ('F', '2016-' + month + '-' + day, record.FEMMINE_16),
        ('F', '2017-' + month + '-' + day, record.FEMMINE_17),
        ('F', '2018-' + month + '-' + day, record.FEMMINE_18),
        ('F', '2019-' + month + '-' + day, record.FEMMINE_19),
        ('F', '2020-' + month + '-' + day, record.FEMMINE_20),
    )
    for item in record_ext:
        records.append(record_base + item)
    # if len(records) > 100:
    #    break
table = pd.DataFrame(records, columns=[
    'region', 'province', 'comune', 
    'age', 'sex', 'date', 'deaths'])

table = table.loc[table['date'].map(date_is_valid)]

In [ ]:
subset = table.groupby(['date'])['deaths'].sum().reset_index().sort_values('date')
subset['days'] = subset['date'].map(days_from_jan_1st)
subset['year'] = subset['date'].map(lambda date:int(date[:4]))
groups = subset.groupby('year')
figure, axes = plt.subplots(1, 1)
axes.set_xlim(0, 90)
axes.set_ylim(0, 1200)
axes.grid(axis='y',zorder=0)
axes.set_xlabel('Days since Jan 1st')
axes.set_ylabel('Nationwide daily deaths from all causes')
axes2 = axes.twiny()
axes2.set_xlim(0, 90)
axes2.set_ylim(0, 1200)
axes2.set_xticks(tuple(numpy.cumsum([0, 31, 28, 31])))
axes2.set_xticklabels(['Jan 1st', 'Feb 1st', 'Mar 1st', 'Apr 1st'])
axes2.grid(zorder=0)
for key, group in groups:
    axes2.plot(group['days'], group['deaths'], 
              label=key, linewidth=1)
axes2.legend()
plt.savefig('figure.png', dpi=300)
plt.show(figure)
plt.close(figure)